In [7]:
import os
import random

from PIL import Image
from typing import Tuple

In [8]:
SEED = 20
random.seed(SEED)

data_folder = "data_object_image_2/training/image_2"
label_folder = "data_object_label_2/training/label_2"

TRAIN_SPLIT = 0.7
VAL_SPLIT = 0.15
# TEST_SPLIT = 0.15

In [9]:
class KittiLabel:
    def __init__(self, label_string):
        label_parts = label_string.strip().split(' ')
        
        self.class_label = label_parts[0]
        self.truncation = float(label_parts[1])
        self.occlusion = int(label_parts[2])
        self.alpha = float(label_parts[3])
        self.bbox = [float(coord) for coord in label_parts[4:8]]
        self.dimensions = [float(dimension) for dimension in label_parts[8:11]]
        self.location = [float(coord) for coord in label_parts[11:14]]
        self.rotation = float(label_parts[14])
        
    def to_string(self):
        bbox_str = ' '.join([f'{coord:.2f}' for coord in self.bbox])
        dimensions_str = ' '.join([str(dimension) for dimension in self.dimensions])
        location_str = ' '.join([str(coord) for coord in self.location])

        label_str = f"{self.class_label} {self.truncation:.2f} {self.occlusion} {self.alpha:.2f} {bbox_str} {dimensions_str} {location_str} {self.rotation:.2f}\n"
        return label_str
    
    def resize(self, prev_dimensions, new_dimensions, padding_tuple):
        x_ratio = new_dimensions[0] / prev_dimensions[0]
        y_ratio = new_dimensions[1] / prev_dimensions[1]

        self.bbox[0] *= x_ratio
        self.bbox[1] *= y_ratio
        self.bbox[2] *= x_ratio
        self.bbox[3] *= y_ratio
        
        self.bbox[0] += padding_tuple[0]
        self.bbox[1] += padding_tuple[1]
        self.bbox[2] += padding_tuple[0]
        self.bbox[3] += padding_tuple[1]

In [10]:
def get_directory_name(dir_type: str, 
                       new_size: Tuple[int, int] = (512, 512), 
                       padding: bool = True):
    folder_name = dir_type
    if new_size:
        folder_name += f"_size_{new_size[0]}_{new_size[1]}"
    else:
        folder_name += "_size_original"
    if padding and new_size:
        folder_name += "_padded"
    else:
        folder_name += "_not_padded"
    folder_name += f"_seed_{SEED}"
    return folder_name


def setup_resized(new_size=(512,512), padding=True, image_file=None, dir_name=None):
    
    # check args
    if not image_file or not dir_name:
        raise ValueError("Image file and target directory not provided")
    os.makedirs(dir_name, exist_ok=True)

    # load image
    image = Image.open(image_file)    
    old_size = image.size
    
    # resize if needed
    if new_size[0] < old_size[0] or padding is False:
        image = image.resize((new_size[0], image.size[1]), resample=Image.BILINEAR)
    if new_size[1] < old_size[1] or padding is False:
        image = image.resize((image.size[0], new_size[1]), resample=Image.BILINEAR)
        
    # pad if needed
    if padding:
        pad_width = max(new_size[0] - old_size[0], 0)
        pad_height = max(new_size[1] - old_size[1], 0)
        padding_tuple = (pad_width // 2, pad_height // 2)
    else:
        padding_tuple = None
    
    # final output image
    if padding_tuple:
        output_image = Image.new("RGB", new_size)
        output_image.paste(image, padding_tuple)
    else:
        output_image = image
        
    # save image
    output_path = os.path.join(dir_name, os.path.basename(image_file))
    output_image.save(output_path)
    
    # get label for image
    label_path = os.path.join(label_folder, os.path.splitext(os.path.basename(image_file))[0] + ".txt")
    if not os.path.exists(label_path):
        raise FileNotFoundError(f"Label not found: {label_path}")
        
    with open(label_path, "r") as path:
        labels = path.readlines()
    
    # resize label
    resized_labels = []
    for label in labels:
        kitti_label = KittiLabel(label)
        kitti_label.resize(old_size, image.size, padding_tuple)
        resized_labels.append(kitti_label.to_string())
    
    resized_label_file = os.path.join(dir_name, os.path.basename(label_path))
    with open(resized_label_file, "w") as output_label:
        output_label.writelines(resized_labels)
        
        
def setup_directories(new_size=(512,512), padding=True):
    
    # load and shuffle
    image_files = sorted([file for file in os.listdir(data_folder) if file.endswith('.png')])
    
    random.seed(SEED)
    random.shuffle(image_files)
    
    # count images
    total_images = len(image_files)
    train_count = int(TRAIN_SPLIT * total_images)
    val_count = int(VAL_SPLIT * total_images)
    
    # get dir names
    train_dir = get_directory_name("train", new_size, padding)
    val_dir = get_directory_name("val", new_size, padding)
    test_dir = get_directory_name("test", new_size, padding)
    
    # create dirs
    try:
        os.makedirs(train_dir, exist_ok=False)
        os.makedirs(val_dir, exist_ok=False)
        os.makedirs(test_dir, exist_ok=False)
    except OSError as error:
        print("Directories already present")
        return

    # copy files to dirs
    for i, image_file in enumerate(image_files):
        if i < train_count:
            setup_resized(new_size, padding, os.path.join(data_folder, image_file), train_dir)
        elif i < train_count + val_count:
            setup_resized(new_size, padding, os.path.join(data_folder, image_file), val_dir)
        else:
            setup_resized(new_size, padding, os.path.join(data_folder, image_file), test_dir)

In [11]:
setup_directories(new_size=(512, 512), padding=True)

In [12]:
setup_directories(new_size=(1024, 1024), padding=True)